#### Note: Latest May 21 (7.30pm) - OneHot Encoding (not...still get_dummies). Creating Kelly's 3 breed categories



## Reading in data and taking a quick look at some charts

In [24]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [25]:
# Read in csv
df = pd.read_csv("Austin_Animal_Center_Outcomes_ORIGINAL.csv")

In [26]:
df.head()

,AnimalID,Name,DateTime,MonthYear,DateofBirth,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,A776359,Gizmo,07/18/2018 04:02:00 PM,07/18/2018 04:02:00 PM,7/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
1,A720371,Moose,02/13/2016 05:59:00 PM,02/13/2016 05:59:00 PM,10/8/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
2,A765349,Einstein,6/8/2018 13:04,6/8/2018 13:04,01/18/2009,Adoption,Foster,Dog,Neutered Male,9 years,Chihuahua Shorthair Mix,Tricolor
3,A760697,Star,10/26/2017 03:22:00 PM,10/26/2017 03:22:00 PM,10/23/2007,Transfer,Partner,Dog,Intact Male,10 years,Yorkshire Terrier Mix,Brown/Black
4,A767231,Millie,02/25/2018 05:19:00 PM,02/25/2018 05:19:00 PM,02/25/2017,Return to Owner,NaN,Dog,Spayed Female,1 year,Jack Russell Terrier/Chihuahua Shorthair,White/Tan


In [27]:
# Drop unnessary columns
df = df.drop(columns=['AnimalID', 'Name', 'DateTime', 'MonthYear', 'DateofBirth', 'OutcomeSubtype', 'AnimalType'])

In [28]:
df2 = df[["OutcomeType", 'SexuponOutcome']]

In [29]:
df2.head()
df2.shape

(66932, 2)

In [30]:
# Drop rows where Outcome_Type is null

df2 = df2.loc[df2.OutcomeType.notna(), ["OutcomeType", "SexuponOutcome"]]

In [32]:
# Check for more nulls
df2.head()
df2.apply(lambda x: sum(x.isnull()/len(df2)))
# df2.shape

OutcomeType       0.00000
SexuponOutcome    0.00003
dtype: float64

In [33]:
# Have to fix the following columns "Sex"
df2 = df2.dropna(axis=0, subset=["SexuponOutcome"])

In [35]:
# Verify that nulls from "Fixed" and "Sex" have been dropped
df2.apply(lambda x: sum(x.isnull()/len(df2)))
df2.shape

(66929, 2)

In [36]:
# # df2.groupby('Outcome_Type').count()
# df2.groupby('Color').count().sort_values('Outcome_Type', ascending=False)
df2.groupby('OutcomeType').count()

,SexuponOutcome
OutcomeType,
Adoption,31405
Died,242
Disposal,25
Euthanasia,1734
Missing,29
Return to Owner,18481
Rto-Adopt,498
Transfer,14515


In [37]:
# Drop all rows 'Return to Owner' ???  Assuming that chip is checked at intake

df3 = df2.drop(df2[df2.OutcomeType == "Return to Owner"].index)

In [38]:
# Verify shape
df3.shape

(48448, 2)

In [40]:
# Verify group by counts (Return to Owner should be gone)
df3.groupby('OutcomeType').count()

,SexuponOutcome
OutcomeType,
Adoption,31405
Died,242
Disposal,25
Euthanasia,1734
Missing,29
Rto-Adopt,498
Transfer,14515


In [97]:
# Anymore missing values before we move on?  If zero, then we are good.
df3.apply(lambda x: sum(x.isnull()/len(df3)))

OutcomeType       0.0
SexuponOutcome    0.0
Binary_outcome    0.0
dtype: float64

In [99]:
df3.groupby(['OutcomeType','SexuponOutcome']).count()

Binary_outcome
OutcomeType SexuponOutcome                
Adoption    Intact Female              419
            Intact Male                399
            Neutered Male            15737
            Spayed Female            14848
            Unknown                      2
Died        Intact Female               74
            Intact Male                 78
            Neutered Male               38
            Spayed Female               22
            Unknown                     30
Disposal    Intact Female                7
            Intact Male                 11
            Neutered Male                3
            Spayed Female                1
            Unknown                      3
Euthanasia  Intact Female              410
            Intact Male                495
            Neutered Male              498
            Spayed Female              292
            Unknown                     39
Missing     Intact Female                7
            Intact Male                 12
            Neutered Male                5
            Spayed Female                5
Rto-Adopt   Intact Female               19
            Intact Male                 28
            Neutered Male              271
            Spayed Female              180
Transfer    Intact Female             3572
            Intact Male               3275
            Neutered Male             4136
            Spayed Female             3243
            Unknown                    289

## Add new features?
1. New feature? - Mixed/Pure Breed? - COMPLETED!! 
2. Split breeds - COMPLETED!!
3. Color -  We have 333 unique color combinations. Hot encoding "Color" will create additional X features if we didn't do anything.  This would be unlikely to create a good performing model.
    *  Option 1 - We take any values occurring fewer than 300 times and places them into the “other” label. We use this to create a new feature called “Top_colors”.  Doing this would reduce 333 unique colors to 30 unique values - COMPLETED!
    *  Option 2 - Split to primary color vs secondary color? - COMPLETED!
4. Creating age groups - COMPLETED!!
5. Breeds - Hot encoding "Breed" will create additional 1,380 new features if we didn't do anything.  
6. New feature? - Single color/Multi color?

#### Creating a Mix/Pure Breed column - As of May 20 9pm, MIX doesn't do anything cause assuming all breeds are mixed.
#### Creating Kelly's 3 breed categories: 0 = mix, secondary not listed ; 1 = secondary breed listed; 2 = purebred

1. Create new MIX column (Kelly's 3 categories)
2. Split breed into breed_primary and breed_second


#### solution - Creating a Breed_category column AND Split into Breed_primary/Breed_second
* 0 = Breed has the word "Mix"
* 1 = If there are 2 breeds listed
* 2 = Only 1 breed in the primary (i.e. None in secondary OR Pure)

In [42]:
# # Make a copy and add new column called Breed_category, set all to PURE
# df4 = df3.copy()
# df4["Breed_category"]= "Prob_pure"
# df4.head()

In [43]:
# df4.shape

In [44]:
# df4.loc[200:210, :]

In [45]:
# # Split Primary and Secondary
# df4[['Breed_primary','Breed_second']] = df4.Breed.str.split("/", n=1, expand=True) 

In [18]:
# df4.loc[200:210, :]

,Outcome_Type,Sex,Age_month,Breed,Color,Breed_category,Breed_primary,Breed_second
200,Transfer,Female,0.2,Dachshund,Tan,Prob_pure,Dachshund,None
201,Transfer,Male,0.2,Dachshund,Tan,Prob_pure,Dachshund,None
205,Transfer,Female,0.2,German Shepherd/Rottweiler,Brown/Black,Prob_pure,German Shepherd,Rottweiler
206,Transfer,Female,0.2,Labrador Retriever/Pit Bull,Brown Brindle,Prob_pure,Labrador Retriever,Pit Bull
208,Transfer,Female,0.2,Rat Terrier Mix,White,Prob_pure,Rat Terrier Mix,None
209,Transfer,Female,0.2,Labrador Retriever/Pit Bull,Brown Brindle/White,Prob_pure,Labrador Retriever,Pit Bull
210,Transfer,Male,0.2,Rat Terrier Mix,White,Prob_pure,Rat Terrier Mix,None


In [46]:
# # Find rows containing word "Mix" and set to 0
# df4.loc[df4["Breed"].str.contains("Mix"), "Breed_category"] = "Mix"

In [47]:
# df4.loc[200:210, :]

In [48]:
# # Find rows containing word "/" and set to 1
# df4.loc[df4["Breed"].str.contains("\/"), "Breed_category"] = "Two"

In [49]:
# df4.loc[200:210, :]

In [50]:
# breed_counts = df4['Breed_primary'].value_counts()
# breed_counts

In [51]:
# df4.loc[200:210, :]

In [52]:
# breed_others = set(breed_counts[breed_counts < 500].index)
# df4['Top_breed'] = df4['Breed_primary'].replace(list(breed_others), 'Others')
# # print(df4['Top_breed'].nunique())


In [53]:
# df4.loc[200:210, :]

# QUESTION!  Should you not use breed_second?

#### Handling Color - Option 1 (Grouping colors)
* How does option1 effect RF/LogReg models?


In [54]:
# # How many unique color combinations?  - 333
# color_counts = df4['Color'].value_counts()
# # len(color_counts)
# print(df4['Color'].nunique())
# color_counts.head(20)

In [55]:
# # # Handling "Color" Option 1
# color_others = set(color_counts[color_counts < 1200].index)
# df4['Top_colors'] = df4['Color'].replace(list(color_others), 'Others')
# print(df4['Top_colors'].nunique())

In [56]:
# df4.head()


#### Creating age categories
* Baby: <=6months
* Young: 7months-24 months
* Adult: 2 years-7years
* Senior: 7+

In [57]:
# df4.head()

In [58]:
# df4['Pet_age_category'] = pd.cut(x=df4['Age_month'], bins=[0, 6, 24, 84,240], labels=['Baby', 'Young', 'Adult', 'Senior'])

In [59]:
# df4.loc[50000:50020, :]
# df4.tail()

In [60]:
# # What's the breakdown by Pet_age_category after binning??
# df4.Pet_age_category.value_counts().sort_values(ascending=False).head(10)

## Changing to Binary Outcome - Take the dog or Don't take the dog?
* Positive Outcome is Adoption - Take in the dog
* Negative Outcome is a combination of Transfer, Euthanized and Death categories - Don't take in the dog

In [61]:
df3.head()

,OutcomeType,SexuponOutcome
0,Adoption,Neutered Male
1,Adoption,Neutered Male
2,Adoption,Neutered Male
3,Transfer,Intact Male
7,Adoption,Spayed Female


In [62]:
# Create new BINARY_OUTCOME column and set to 0

df3["Binary_outcome"]=0
df3.head()

,OutcomeType,SexuponOutcome,Binary_outcome
0,Adoption,Neutered Male,0
1,Adoption,Neutered Male,0
2,Adoption,Neutered Male,0
3,Transfer,Intact Male,0
7,Adoption,Spayed Female,0


In [63]:
# Find rows containing word "Adoption" and set to 1 (for Positive Outcome)
# All other rows will be 0 (for Negative Outcome)

df3.loc[df3["OutcomeType"].str.contains("Adoption"), "Binary_outcome"] = 1

In [65]:
df3.loc[200:300, :]

,OutcomeType,SexuponOutcome,Binary_outcome
200,Adoption,Spayed Female,1
201,Adoption,Neutered Male,1
202,Adoption,Neutered Male,1
204,Transfer,Spayed Female,0
205,Transfer,Intact Female,0
...,...,...,...
292,Adoption,Spayed Female,1
293,Adoption,Neutered Male,1
294,Adoption,Neutered Male,1
295,Adoption,Neutered Male,1


## Still get_dummies

In [66]:
df3.head()

,OutcomeType,SexuponOutcome,Binary_outcome
0,Adoption,Neutered Male,1
1,Adoption,Neutered Male,1
2,Adoption,Neutered Male,1
3,Transfer,Intact Male,0
7,Adoption,Spayed Female,1


In [69]:
# Drop these columns:
# Outcome_Type column since we now have Binary_outcome 
# Age_month since we have Pet_age_category
# Breed since we have Breed_primary, Breed_second     - SHOULD BE DROP BREED_SECOND??
# Color since we have Color1, Color2
df4 = df3.drop(columns=['OutcomeType'])
df4.head()

,SexuponOutcome,Binary_outcome
0,Neutered Male,1
1,Neutered Male,1
2,Neutered Male,1
3,Intact Male,0
7,Spayed Female,1


## Cleaning it up - STOP!!!!

In [70]:
# V1 - ALl in  
# df6 = pd.get_dummies(df5[['Sex', 'Breed_category','Top_breed', 'Top_colors', 'Pet_age_category', 'Binary_outcome']])

# V2 - Only Sex and Age
df5 = pd.get_dummies(df4[['SexuponOutcome', 'Binary_outcome']])

In [71]:
df5.columns

Index(['Binary_outcome', 'SexuponOutcome_Intact Female',
       'SexuponOutcome_Intact Male', 'SexuponOutcome_Neutered Male',
       'SexuponOutcome_Spayed Female', 'SexuponOutcome_Unknown'],
      dtype='object')

In [73]:
# Define X and y
X = df5.drop(['SexuponOutcome_Intact Female', 'Binary_outcome'], axis="columns")
y = df5[['Binary_outcome']]

In [46]:
X

,Pet_age_category_Baby,Pet_age_category_Young,Pet_age_category_Adult
0,1,0,0
1,1,0,0
2,1,0,0
4,1,0,0
6,1,0,0
...,...,...,...
66772,0,0,0
66774,0,0,0
66786,0,0,0
66787,0,0,0


In [74]:
y

,Binary_outcome
0,1
1,1
2,1
3,0
7,1
...,...
66926,1
66927,1
66928,1
66929,1


## Random Forest model


In [75]:
# Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [76]:
# Import, initialize, fit and predict
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators = 100, random_state = 42) 
rf_model.fit(X_train, y_train)
predict_y_test = rf_model.predict(X_test)

C:\Users\Lenovo\Anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [77]:
# Validate - run accuracy score
from sklearn import metrics
print("Accuracy score: ", metrics.accuracy_score(y_test, predict_y_test))

Accuracy score:  0.8020970937912814


In [78]:
# RF Feature importance?
feature_list = list(X.columns)
feature_importance = pd.Series(rf_model.feature_importances_, index=feature_list).sort_values(ascending=False) 
print(feature_importance[0:10])

SexuponOutcome_Neutered Male    0.411671
SexuponOutcome_Intact Male      0.284252
SexuponOutcome_Spayed Female    0.281975
SexuponOutcome_Unknown          0.022102
dtype: float64


In [79]:
# Validate with cross validation

from sklearn.model_selection import cross_val_score

scores = cross_val_score(rf_model, X, y, cv=10, scoring="accuracy")
print(scores)

C:\Users\Lenovo\Anaconda3\envs\pythondata\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Lenovo\Anaconda3\envs\pythondata\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Lenovo\Anaconda3\envs\pythondata\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Lenovo\Anaconda3\envs\pythondata\lib\site-packages\sklearn\model_selec

[0.80928793 0.80412797 0.80557276 0.80433437 0.79380805 0.80268318
 0.81031992 0.79731682 0.79830718 0.8104872 ]


In [80]:
scores.mean()

0.8036245365198104

## Logistic Regression

In [81]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [82]:
from sklearn.linear_model import LogisticRegression
logReg = LogisticRegression()
logReg.fit(X_train, y_train)

C:\Users\Lenovo\Anaconda3\envs\pythondata\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [83]:
print(f"Training Data Score: {logReg.score(X_train, y_train)}")
print(f"Testing Data Score: {logReg.score(X_test, y_test)}")

Training Data Score: 0.8041336415675914
Testing Data Score: 0.8020970937912814


## Naive Bayes model


In [84]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [85]:
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
nb_model = GaussianNB()

# Train the model using the training sets
nb_model.fit(X_train, y_train)

#Predict Output
y_predicted= nb_model.predict(X_test)


C:\Users\Lenovo\Anaconda3\envs\pythondata\lib\site-packages\sklearn\naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [86]:
print(y_predicted)

[1 1 1 ... 1 1 1]


In [87]:
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_predicted))

Accuracy: 0.7237450462351387


## SVM


In [89]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=11)  

target = df5["Binary_outcome"]



In [90]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train, y_train)

C:\Users\Lenovo\Anaconda3\envs\pythondata\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [91]:
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=11, shrinking=True,
  tol=0.001, verbose=False)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=11, shrinking=True, tol=0.001,
    verbose=False)

In [92]:
# Model Accuracy
print('Test Acc: %.3f' % model.score(X_test, y_test))

Test Acc: 0.804


In [93]:
 # Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.91      0.49      0.64      4268
           1       0.78      0.97      0.87      7844

    accuracy                           0.80     12112
   macro avg       0.84      0.73      0.75     12112
weighted avg       0.83      0.80      0.78     12112

